In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
print("1")

1


In [1]:
!pip install -U albumentations
!pip install opencv-python-headless==4.1.2.30 
!pip install numpy==1.19.3
!pip install matplotlib

  Using cached numpy-1.22.2-cp38-cp38-win_amd64.whl (14.7 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 0.17.3 requires matplotlib==3.3.3, which is not installed.
d2l 0.17.3 requires numpy==1.18.5, but you have numpy 1.22.2 which is incompatible.


  Using cached numpy-1.19.3-cp38-cp38-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.2
    Uninstalling numpy-1.22.2:
      Successfully uninstalled numpy-1.22.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 0.17.3 requires matplotlib==3.3.3, which is not installed.
imageio 2.16.1 requires numpy>=1.20.0, but you have numpy 1.19.3 which is incompatible.
d2l 0.17.3 requires numpy==1.18.5, but you have numpy 1.19.3 which is incompatible.


  Using cached matplotlib-3.5.1-cp38-cp38-win_amd64.whl (7.2 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 0.17.3 requires matplotlib==3.3.3, but you have matplotlib 3.5.1 which is incompatible.
d2l 0.17.3 requires numpy==1.18.5, but you have numpy 1.19.3 which is incompatible.


In [2]:
!nvidia-smi

Tue Mar  1 11:10:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P8     4W /  N/A |    340MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# 作者:ZhaoYu Wang
# 日期:2022年02月22日
import os
import re

import cv2
import matplotlib  
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.models.detection
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

ROOT_DIR = ""

IMAGE_SIZE = 900


In [4]:
def load_images(path):
    images = []
    image_paths = []
    direciton_list = os.listdir(path)
#     print(direciton_list)
#     print(len(direciton_list))
    image_shape_list = []
    print("reading images......")
    for imgfile in direciton_list:
      image_path = path + imgfile
      # print(image_path)
      img = cv2.imread(image_path)
      # print(imgfile)
      img_shape = (img.shape[0], img.shape[1])
      images.append(img)
      image_paths.append(image_path)

      del img
      image_shape_list.append(img_shape)

    image_ids = [i for i in range(len(images))]
    print("reading done")
    return image_paths, image_ids, image_shape_list


In [5]:
class TextDataset(Dataset):
    def __init__(self, dataframe, transforms=None):
        super().__init__()
        self.dataframe = dataframe
        self.transforms = transforms
        self.length = len(dataframe)
        self.image_ids = dataframe['image_id'].unique()

    def __getitem__(self, item: int):
        def get_boxes(path, img_shape):
            # print(img_shape)
            boxes = []
            label_path = ROOT_DIR + "labels/" + path.split('/')[-1].split('.')[0] + ".txt"
            # print(label_path)
            text_file = open(label_path, "r")
            for line in text_file:
                info_list = line.split(",")[:9]
                # print(info_list)
                x1, y1, x2, y2, x3, y3, x4, y4 = map(int, info_list[:8])
                if max(y1, y2, y3, y4)>600:
                  print(path)
                x_min = min(x1, x2, x3, x4) * (IMAGE_SIZE / img_shape[1])
                x_max = max(x1, x2, x3, x4) * (IMAGE_SIZE / img_shape[1])
                y_min = min(y1, y2, y3, y4) * (IMAGE_SIZE / img_shape[0])
                y_max = max(y1, y2, y3, y4) * (IMAGE_SIZE / img_shape[0])
                # if 600*1.205 == y_max:
                #   print(path)
                # print([x_min, y_min, x_max, y_max])
                boxes.append([x_min, y_min, x_max, y_max])
            return np.array(boxes)

        image_id = self.image_ids[item]
        path = self.dataframe.iloc[item].path

        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255

        # plt.imshow(image)
        # plt.show()

        boxes = get_boxes(path, self.dataframe.iloc[item].image_shape)
        # print(boxes)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([item])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id,path

    def __len__(self) -> int:
        return self.image_ids.shape[0]


In [6]:
class TestTextDataset(Dataset):
    def __init__(self, dataframe, transforms=None):
        super().__init__()
        self.dataframe = dataframe
        self.transforms = transforms
        self.length = len(dataframe)
        self.image_ids = dataframe['image_id'].unique()

    def __getitem__(self, item: int):
        image_id = self.image_ids[item]
        path = self.dataframe.iloc[item].path

        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255


        if self.transforms:
            sample = {
                'image': image
            }
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id,path

    def __len__(self) -> int:
        return self.image_ids.shape[0]




In [7]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


In [8]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_test_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ])


def collate_fn(batch):
    return tuple(zip(*batch))



In [8]:
# torch.cuda.empty_cache()
# image_paths, image_ids, image_shape_list = load_images(ROOT_DIR + "/images/")
# dataframe = pd.DataFrame(np.array(image_paths), columns=['path'])
# dataframe['image_id'] = image_ids
# dataframe['image_shape'] = image_shape_list
# # print(dataframe.head())
# print(dataframe['image_id'].unique().shape[0])  # 626

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# num_classes = 2  # 1 class (wheat) + background
# # get number of input features for the classifier
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# # replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# # 划分训练集和验证集
# split_point = int(dataframe['image_id'].unique().shape[0] * 0.75)
# # print("划分训练集和验证集点", split_point)

# train_df = dataframe.iloc[:split_point]
# # valid_df = dataframe.iloc[split_point:-3]
# valid_df = dataframe.iloc[split_point:]
# # test_df = dataframe.iloc[-3:]

# train_dataset = TextDataset(train_df, get_train_transform())
# valid_dataset = TextDataset(valid_df, get_valid_transform())
# test_dataset = TextDataset(test_df, get_valid_transform())

# # split the dataset in train and test set
# indices = torch.randperm(len(train_dataset)).tolist()
# train_data_loader = DataLoader(
#     train_dataset,
#     batch_size=16,
#     shuffle=False,
#     num_workers=4,
#     collate_fn=collate_fn
# )
# valid_data_loader = DataLoader(
#     valid_dataset,
#     batch_size=8,
#     shuffle=False,
#     num_workers=4,
#     collate_fn=collate_fn
# )
# test_data_loader = DataLoader(
#     test_dataset,
#     batch_size=8,
#     shuffle=False,
#     num_workers=4,
#     collate_fn=collate_fn
# )

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# torch.cuda.empty_cache()
# images, targets, image_ids ,images_paths= next(iter(train_data_loader))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
# boxes = targets[2]['boxes'].cpu().numpy().astype(np.int32)
# # print(type(images[2]))
# sample = images[2].permute(1,2,0).cpu().numpy()
# # print(sample)

# fig, ax = plt.subplots(1, 1)
# # ax.imshow(sample)
# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 1)
# # plt.imshow(sample)
# # ax.imshow(sample)
# # plt.savefig('pic1.jpg', bbox_inches='tight', dpi=1000)

# # plt.show()

# torch.cuda.empty_cache()
# model.to(device)
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer  = torch.optim.SGD(params,lr = 0.005,momentum=0.9,weight_decay=0.0005)
# lr_scheduler=None



In [9]:
def get_faster_rcnn_model_v2(model):
    new_num_class = 10
    model_new = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        num_classes=new_num_class)
    
    pretrained_state_dict = model.state_dict()
    change_keys = [
        'roi_heads.box_predictor.cls_score.weight',
        'roi_heads.box_predictor.cls_score.bias',
        'roi_heads.box_predictor.bbox_pred.weight',
        'roi_heads.box_predictor.bbox_pred.bias'
    ]
    
    
    from_scratch_state_dict = model_new.state_dict()
    
   
    for change_key in change_keys:
        pretrained_state_dict[change_key] = from_scratch_state_dict[change_key]
    
    model_new.load_state_dict(pretrained_state_dict)
    return model_new

In [22]:
torch.cuda.empty_cache()

In [10]:
!nvidia-smi

Tue Mar  1 11:11:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8     4W /  N/A |    340MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
#预训练
torch.cuda.empty_cache()
image_paths, image_ids, image_shape_list = load_images(ROOT_DIR + "images/")
dataframe = pd.DataFrame(np.array(image_paths), columns=['path'])
dataframe['image_id'] = image_ids
dataframe['image_shape'] = image_shape_list
# print(dataframe.head())
print(dataframe['image_id'].unique().shape[0])  # 626




model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
num_classes = 2  # 1 class (wheat) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# Load the trained weights
model.load_state_dict(torch.load('fasterrcnn_resnet50_fpn_epoch_3.pth'))

# model = get_faster_rcnn_model_v2(model)


# 划分训练集和验证集
split_point = int(dataframe['image_id'].unique().shape[0] * 0.75)
train_df = dataframe.iloc[:split_point]
# valid_df = dataframe.iloc[split_point:-3]
valid_df = dataframe.iloc[split_point:-3]
test_df = dataframe.iloc[-3:]

train_dataset = TextDataset(train_df, get_train_transform())
valid_dataset = TextDataset(valid_df, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()
train_data_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

torch.cuda.empty_cache()
images, targets, image_ids ,images_paths= next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
#print(type(images[1]))
sample = images[1].permute(1,2,0).cpu().numpy()
# print(sample)

fig, ax = plt.subplots(1, 1)
# ax.imshow(sample)
for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 1)
# plt.imshow(sample)
# ax.imshow(sample)
# plt.savefig('pic1.jpg', bbox_inches='tight', dpi=1000)

# plt.show()

torch.cuda.empty_cache()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer  = torch.optim.SGD(params,lr = 0.005,momentum=0.9,weight_decay=0.0005)
lr_scheduler=None


reading images......
reading done
1000


In [18]:
# Train

num_epochs = 50

loss_hist = Averager()
valid_loss_hist = Averager()
itr = 1
loss_list = []
valid_loss_list = []
torch.cuda.empty_cache()

for epoch in range(num_epochs):
    loss_hist.reset()
    valid_loss_hist.reset()
    torch.cuda.empty_cache()

    #Train
    model.train()
    for images ,targets,image_ids,paths in train_data_loader:
        torch.cuda.empty_cache()
        
        images  = list(image.float().to(device) for image in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]
        
        
        loss_dict = model(images,targets)

        losses  =sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        


D:\development_tools\Anaconda3\envs\limuPytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: Found dtype Double but expected Float

In [24]:
# Train

num_epochs = 50

loss_hist = Averager()
valid_loss_hist = Averager()
itr = 1
loss_list = []
valid_loss_list = []
torch.cuda.empty_cache()

for epoch in range(num_epochs):
    loss_hist.reset()
    valid_loss_hist.reset()
    torch.cuda.empty_cache()

    #Train
    model.train()
    for images ,targets,image_ids,paths in train_data_loader:
        torch.cuda.empty_cache()
        
        images  = list(image.float().to(device) for image in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]
        
        
        loss_dict = model(images,targets)

        losses  =sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        itr+=1

    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss:{loss_hist.value}")
    loss_list.append(loss_hist.value)

    # Validate
    model.eval()
    for images ,targets,image_ids ,image_paths in valid_data_loader:
        torch.cuda.empty_cache()
        images =list(image.to(device) for image in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]
        with torch.no_grad():
            loss_dict = model(images)
            for x in range(len(loss_dict)):
                if len(loss_dict[x]['scores'])>0:
                    losses = ((sum((1-loss)**2 for loss in loss_dict[x]['scores'])))/len(loss_dict[x]['scores'])
                    loss_value = losses.item()

                    valid_loss_hist.send(loss_value)
    print(f"Epoch #{epoch} val_loss: {valid_loss_hist.value}")
    valid_loss_list.append(valid_loss_hist.value)
    torch.save(model.state_dict(), f'models/fasterrcnn_resnet50_fpn_epoch_{epoch}.pth')

RuntimeError: Found dtype Double but expected Float

In [15]:
!nvidia-smi

Mon Feb 28 22:32:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:06.0 Off |                    0 |
| N/A   41C    P0    27W /  70W |   4282MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
figs, axes = plt.subplots(1, 1, figsize=(20, 5))
axes.plot(range(50), loss_list, 'r-', label='Train Loss')
axes.plot(range(50), valid_loss_list, 'b-', label='Val Loss')
axes.legend()
axes.set_xlabel("Epoch")
axes.set_ylabel("Loss")

plt.show()
figs.savefig('loss.png', bbox_inches='tight')

NameError: ignored

In [12]:
from torchvision import transforms as torchtrans
from matplotlib import patches

In [13]:
def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig,a =plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x,y ,width,height = box[0],box[1],box[2]-box[0],box[3]-box[1]
        rect = patches.Rectangle((x,y),
                                 width,height,
                                 linewidth=1,
                                 edgecolor='r',
                                 facecolor='none'
                                 )
        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')

def apply_nms(orig_prediction, iou_thresh=0.3):
    keep = torchvision.ops.nms(orig_prediction['boxes'],orig_prediction['scores'],iou_thresh)
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    return final_prediction

In [ ]:
images, targets, image_ids ,image_paths= next(iter(valid_data_loader))

images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

boxes = targets[2]['boxes'].cpu().numpy().astype(np.int32)
sample = images[2].permute(1,2,0).cpu().numpy()

model.eval()

# <================ Non max suppression =======================>

indices = torch.randperm(len(valid_dataset)).tolist()
dataset_test = torch.utils.data.Subset(valid_dataset, indices[-10:])
ig, tg, ig_id ,ig_path= dataset_test[5]
with torch.no_grad():
  
    prediction = model([ig.to(device)])[0]

# print("1111")
plot_img_bbox(torch_to_pil(ig), tg)
# print("3333")
# print(prediction)
plot_img_bbox(torch_to_pil(ig), prediction)

cpu_device = torch.device("cpu")
nms_prediction = apply_nms(prediction, iou_thresh=0.1)
print('NMS APPLIED MODEL OUTPUT')

# print(nms_prediction)
plot_img_bbox(torch_to_pil(ig), nms_prediction)

# <================ Non max suppression =======================>


outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [16]:
import time
import random
time.time()

1645946394.6290905

In [15]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return pred_strings

In [17]:
def plot_origin_img_bbox(img_id, img_path, target):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1, 1)
    fig.set_size_inches(20, 20)
    plt.imshow(img)
    currentAxis=plt.gca()
    for box in (target['boxes']):
        x, y = box[0] * (img.shape[1] / IMAGE_SIZE), box[1] * (img.shape[0] / IMAGE_SIZE)
        width, height = (box[2] - box[0]) * (img.shape[1] / IMAGE_SIZE), (box[3] - box[1]) * (img.shape[0] / IMAGE_SIZE)
        padding_width, padding_height = width * 0.1, height * 0.1
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth=1,
                                 edgecolor='r',
                                 facecolor='none'
                                 )
        # Draw the bounding box on top of the image
        currentAxis.add_patch(rect)

    plt.savefig(str(int(time.time()))+str(random.randint(0,100))+".png", dpi=100)
    plt.show()

In [25]:
torch.cuda.empty_cache()

In [18]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2  # 1 class (wheat) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
# Load the trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/models/fasterrcnn_resnet50_fpn_epoch_3.pth'))
model.eval()
x = model.to(device)

test_dataset = TestTextDataset(valid_df, get_test_transform())

test_data_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn

)
detection_threshold = 0.5
results = []
score_list = []
for images, image_ids, image_paths in test_data_loader:

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):
        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()

        score_list.append(np.mean(scores).tolist())

        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        scores = scores[scores >= detection_threshold]

        image_id = image_ids[i]
        path = image_paths[i]

        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores),
            'path':path

        }
        results.append(result)

avg_score = 0.0
c = 0
for i in score_list:
    try:
        avg_score += int(i * 1000)
    except:
        c += 1
avg_score = avg_score / 1000
print("Average scores:", avg_score / (len(score_list) - c))

test_target = []
for t in results:
    tg = {}
    tg['image_id'] = t['image_id']
    tg['image_path'] = t['path']
    boxes = np.array(([list(map(int, b.split()[1:])) for b in t['PredictionString']]))
    boxes = np.reshape(boxes, (len(boxes), 4))
    tg['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
    tg['scores'] = torch.as_tensor([float(list(b.split(' '))[0]) for b in t['PredictionString']],
                                    dtype=torch.float32)
    tg['labels'] = torch.ones((tg['boxes'].shape[0],), dtype=torch.int64)
    test_target.append(tg)

for i in test_target:
    nms_prediction =apply_nms(i,iou_thresh=0.2)
    plot_origin_img_bbox(img_id=i['image_id'],img_path=i['image_path'],target=i)


Average scores: 0.9981048387096775


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
